In [83]:
import pickle
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.stem import porter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
import os
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
%matplotlib inline
plt.style.use('seaborn')

In [84]:
true_df = pickle.load(open("pickle_files/true_df.pkl", "rb" ))


In [85]:
true_df.head()

,body,num_source,source,title
0,The following statements were posted to the ve...,2.0,Reuters,"Trump on Twitter (June 6): Mitch McConnell, Jo..."
1,"TEL AVIV (Reuters) - Donald Trump’s attorney, ...",2.0,Reuters,Giuliani says Mueller's investigators trying t...
2,WASHINGTON (Reuters) - U.S. House of Represent...,2.0,Reuters,House Speaker doubtful of NAFTA passage in cur...
3,BRUSSELS (Reuters) - U.S. Defense Secretary Ji...,2.0,Reuters,"In Europe, Mattis is in familiar spot: calming..."
4,"WASHINGTON/MORRISTOWN, N.J. (Reuters) - A lawy...",2.0,Reuters,Lawyer to ex-Trump campaign aide Manafort taps...


In [86]:
book = [{'body': """Erik Prince, the founder of private security company Blackwater, has found himself embroiled in special counsel Robert Mueller’s investigation of potential coordination between the Trump campaign and the Kremlin.

During the campaign, Prince reportedly met at Trump Tower with Donald Trump Jr., operative George Nader, and social media specialist Joel Zamel to discuss a potential pro-Trump social media influence operation. He also met with Russian sovereign wealth fund manager Kirill Dmitriev during the transition period—a meeting reportedly planned to set up a backchannel between the Trump administration and Russia. Those revelations raise questions about his relationship to the Trump administration—questions Mueller is reportedly investigating.

ADVERTISEMENT


Prince, a billionaire and the brother of Education Secretary Betsy DeVos, has also lobbied for a massive overhaul of the way the Afghanistan War is being prosecuted, pushing for increased reliance on private contractors to train Afghan troops to fight the Taliban. Prince was also recently the subject of an in-depth Washington Post report about the work his security company Frontier Services Group is doing in China. The report quoted critics saying Prince’s work runs counter to American interests in Asia.

This is Prince’s first on-the-record interview in months. It has been lightly edited for clarity.

There’s been a lot of reporting that Mueller’s interested in some of the meetings you had in the lead-up to the campaign and after the election and I was just wondering if you could tell me if you’ve heard from anyone on Mueller’s team?

I certainly understand the intense interest in the investigation and certainly some of the wild-eyed reporting in the media. I have spoken voluntarily to Congress and I also cooperated with the special counsel. I have plenty of opinions about the various investigations but there’s no question some people are taking it seriously and I think it’s best to keep my opinion on that to myself for now. All I will add is that much of the reporting about me in the media is inaccurate, and I am confident that when the investigators have finished their work, we will be able to put these distractions to the side.


You told the House intelligence committee that the Seychelles meeting [with Dmitriev] was unplanned, but ABC reported that George Nader briefed you on it beforehand. What do you make of that ABC reporting?

RELATED IN POLITICS

Boehner: ‘There Is No GOP. There’s a Trump Party.’

The GOP Loves Diplomacy—When It’s Not a Democrat Doing It

GOP Senators Shred Rudy: ‘I Would Hire a New Lawyer’
All I can say is, there’s been a lot of media reporting about me over the years and most of it is wrong. They get it wrong way more than they ever get it right.

ADVERTISING


“I have spoken voluntarily to Congress and I also cooperated with the special counsel.”
— Erik Prince
What do you think the United States’ posture toward Russia should be? Do you support the president’s rhetoric about trying to thaw that relationship?

Absolutely. As I’ve said before, if Franklin Roosevelt can work with Joseph Stalin to defeat German fascism, Nazi fascism, national socialist fascism, then certainly Donald Trump can work with Putin to defeat Islamic fascism. And I think good statesmanship could even start to drive a wedge between Russian policy and Iran policy because we can disagree vehemently on their policy in Ukraine but we don’t have to be, certainly, their enemy in the Middle East. And even from a NATO perspective, I mean look, remember, 400,000 Americans died in World War II. Twenty-two million Russians died breaking the Nazi army. And from a Russian perspective, there are more unfriendly nations aligned on their borders now than at any time since May of 1940. So I don’t think we have to be provocative with NATO and I think it’s a good idea for the president to reach out diplomatically. I mean for heaven’s sakes, he’s sitting down and talking to Kim of North Korea. Putin is a much more rational actor and I think it’s totally appropriate for the president to sit down and try to thaw the situation.

Could you tell me some about what your relationship with this administration is like and particularly if you’ve had any conversations with new national security adviser John Bolton about your Afghanistan proposal?

No, not yet. I’ll tell you, I have just made an op-ed and it’s being submitted to various publications because I’m getting ready to make a big push on that again because the president gave the Pentagon what they wanted, more money, more troops, and what we’ve got is more death and more carnage and clearly we’re not winning. And since the last op-ed [in The New York Times, arguing for a new war strategy focused on using contractors], I’ve certainly done homework and research to the point where the White House was asking last summer for a very detailed concept of operations and budget, how to do this differently and far more cheaply. And so having let that bake for a year, we have a very different plan that would save the taxpayers well north of $40 billion and it would tie off the conventional involvement of the Pentagon from Afghanistan.

GET THE BEAST IN YOUR INBOX!

Enter your email address


By clicking “Subscribe,” you agree to have read the Terms of Use and Privacy Policy
SUBSCRIBE
What do you make of how Defense Secretary Jim Mattis is handling the Afghanistan war?

Well here’s the thing, what worked after 9/11 were a few CIA case officers and special forces guys backed by air power working with the locals. When we went to a conventional Pentagon battle plan, we’ve gone backwards ever since. The Pentagon has largely mirrored the approach of the Soviet army in Afghanistan, with the same results. Secretary Mattis inherits the momentum of a big machine going in one direction, and that machine will continue, like Newton’s first law, an object tends to remain in motion until it’s acted upon by a greater force. Hopefully Donald Trump is that force to change course, to put us on a winning strategy. Look this year the United States will spend $62 billion in Afghanistan, the OCO [overseas contingency operations—the Pentagon’s war funds] and all the money they have for all the overseas basic support for that theater. More than a million dollars a week while we are $21 trillion in debt. And now there are American kids dying there who were toddlers when the Twin Towers came down. We don’t need a multigenerational war in America.

Do you worry that Mattis is a countervailing force against the president’s arguably better instincts on Afghanistan? Do you worry that he is a voice for the status quo?

All I will say on that is that the president asked for options last year and the only options he was given by his then very conventional national security adviser, a three star honor officer and the Pentagon was more money and more troops or pull out. And there wasn’t a whole lot of innovation presented to the president. I’m going to make a hard push again because I think the president was close to listening to an unconventional approach but given it was right around the time of those terrible race riots in Charlottesville and I think the president took a pounding for that and I don’t think he was ready to do something unconventional in Afghanistan. But with the makings of a successful summit in Singapore, perhaps the president’s ready to try a different approach.

“I think the president was close to listening to an unconventional approach but given it was right around the time of those terrible race riots in Charlottesville... I don’t think he was ready to do something unconventional in Afghanistan.”
— Erik Prince
Do you think John Bolton is going to be more open to your proposal than his predecessor H.R. McMaster was?

I think so. If past performance is indicative of future performance, that’s probably the case.

Have you picked up on any signs that the U.S. might be negotiating with the Taliban to wind down the war?

I’m well acquainted with a number of people that are involved in the peace counsel, and to say that you’re negotiating with the Taliban, it’s a very fractured—it’s not like it was under [former Taliban chief] Mullah Omar. And some Taliban units might want to reconcile but others absolutely not. The big issue is, no one has any confidence with the government because if a Taliban unit reconciles, they’re going to immediately need protection from the other Taliban units or from the Daesh guys or even from the ISI [Pakistan’s intelligence agency]. And those guys play for keeps. They don’t wipe out an entire village, tribe or whatever. So I don’t see a lot of those units wanting to do it until the Afghanistan security forces demonstrate showing up reliably with combat power to defend their interests. So when an Afghan unit still gets surrounded and annihilated over a matter of days, that wouldn’t give anyone a lot of confidence and trust in the government’s protection.

The Post had a big story about the work that Frontier Services Group is doing in China [reporting that the firm is training Chinese police and military officials]. They reported that there had been a spate of departures of American executives over the course of the last year because those executives believed that your company’s work in China could conflict with U.S. interests. Is that correct?

I would say they departed for underperformance and, look, there was a huge mischaracterization by the Post. Once again, they get it wrong. FSG has not provided any training to the Chinese military or police or anything like that at all. And this idea where they try to feed this picture of a Blackwater-like facility in China, just think of how ridiculous that is. There’s no firearms ownership in China at all. So the idea of a contractor providing anything in China is not only 100 percent, it’s 10,000 percent impossible.

You were saying that FSG hasn’t done any training of any Chinese police or military personnel. The Post was unequivocal that you guys had. The Post was just completely wrong about that?

The company has a very minority investment in a training facility that perhaps in the past has provided that, but nothing under anyway FSG’s control, auspices whatsoever. FSG is only training its own bodyguard personnel for terrorism avoidance. No firearms, no tactics, nothing like that.

The Post quoted some critics of yours suggesting that it runs counter to American values for you to do the work you’re doing with the Chinese, suggesting that it’s unpatriotic. What do you say to those criticisms?

When the D.C. beltway pundits mock my patriotism, I almost take it as a badge of honor. I’m certainly not doing anything and never have done anything contrary to America’s interests. The only thing FSG does now is provide transportation by road throughout Africa. We haul groceries, we provide medevac [medical evacuation] support and in the future we provide transportation, medevac, or even security for infrastructure, mining, or energy projects invested in or financed by any number of companies, be they Hong Kong, Chinese, Japanese, Korean, or Western companies operating in Africa and Asia.

One thing the Post said is that Frontier has a facility in China’s Xianjiang province. Is that correct?

It’s an office and it’s a place to transfer truck trailers at most. There’s no training base or facility. I think there was a translation problem where people say, oh, you have a base there. It is not a base as any Westerner would imagine. It is basically a logistics yard.

Is FSG doing work connected to the development of the One Belt One Road infrastructure project?

One Belt One Road is a characterization. It’s certainly an effort by China to extend logistics lines to tie into neighbors’ markets more effectively. So if you’re hauling copper ore out of a mine in Africa, you can technically say that’s part of the One Belt One Road. But we haven’t done any new programs that have been announced so technically for now I would say FSG is not. Will they be in the future? It’s possible, it depends on what the bids are for any company, whether you’re bidding to support a Mitsubishi program out of Japan, a Chinese company, or any one of the major oil or mining companies out of the West. But FSG doesn’t exist solely to support One Belt One Road.

Do you worry that One Belt One Road is an effort to undermine U.S. influence?

No, I think that people that trade well together tend to cooperate economically and politically much better than the people that don’t trade. And when I look at the transformative effect that, for example, a railroad when it finally connected in the 1870s America, it took what was a 180-day voyage by foot and horse down to six days. And so for China to put down money to build a highway or a railway across parts of Africa or Asia, it’s a hugely positive transformative effect on people’s lives in those neighboring countries.

I think the United States should shift away from its focus solely on counterterrorism and military spending. And we can certainly spend money a lot smarter on infrastructure programs than we do now, certainly than we’ve done in the last 20 or so years. I’ll throw you a simple comparison, the whole China/Pakistan economic corridor is about $40 billion over eight years. That’s the amount of finance that China’s committed to pipelines, rails, and hydropower. We’re blowing that now as a country in nine months in Afghanistan. And I would say we are wasting it. So if the United States wants to have lasting influence in these places, let’s spend money a little smarter and invest in things that last.

What do you make of the emerging coalition of the Saudis, the Emiratis, Israel, and the United States working together to isolate Iran? Do you think that’s a good thing?

Sure, certainly. The Iranians are really good at subversion and hybrid war, they’ve been doing it really well in Lebanon for decades, they’re doing it in Yemen now, and I think all the regions from the Gulf see Iran trying to behave like a hegemon. And a hegemon with nuclear weapons is even more dangerous. [Iran does not have any nuclear weapons and gave up much of its capacity to develop them under a deal made during the Obama administration—ed.] And I think their economy is not in great shape, their population is upset, and I think most of the population of Iran wants to live a much more free and secular life, and having the nations unite against them, their neighbors, is an overall positive thing.

Going back to the Post’s story, one thing they referred to was the possibility that there’s a DOJ investigation related to aircraft being considered for sale to the Azerbaijanis. What do you make of that whole issue?

I know nothing of that at all.

You’ve advocated for increasing the use of contractors in Afghanistan. Are there any other crisis flashpoints where you think that type of approach would be helpful? Do you see an increased use for contractors dealing with the border crisis or dealing with the war in Yemen?

Let’s leave that one for the next interview. That’s a whole nother article."""}]

In [87]:
book_df = pd.DataFrame.from_dict(book)

In [88]:
stemmer = porter.PorterStemmer()
stopwords = stopwords.words()

In [89]:
def clean_text(text):
    '''Removes stop words and changes word to stem words'''
    cleaned_text = []
    for post in text:
        cleaned_words = []
        for word in post.split():
            low_word = stemmer.stem(word.lower())
            if low_word not in stopwords:
                cleaned_words.append(low_word)
        cleaned_text.append(' '.join(cleaned_words))
    return cleaned_text

In [90]:
def drop_nouns(text_list):
    no_nouns = []
    cleaned_text = []
    phrases = []
    for x in text_list:
        words = pos_tag(word_tokenize(x))
        #werdz2 = (['CC','CD','DT','EX','FW','IN','LS','MD','PDT','POS','PRP','PRP$',
                   #'RP','TO','UH','WDT','WP','WP$','WRB', 'NNP', 'NNPS', 'NN', 'NNS', ',', '.', ':', '(', ')'])
        werdz3 = ['NNP', 'NN', 'NNP', 'NNPs', 'NNS', ',', '.', ':', '(', ')', '#', '``']
        werdz = [s for s in words if s[-1] not in werdz3]
        #print(werdz)
        say_no = ['@','[', ']', 'amp', 'window', 'open','click', 'googletag', 'gpt', 'linkitem', 'googletag', 'getelementbyid',
        'config', 'ldadinit', 'advertis', 'typeof', 'adsdiv', 'fjs', 'js', 'http', 'com', 'awr', 'new', 'function', 'div',
        'ad', 'script', 'typeof', 'nr_is_logged_in', 'undefined', 'adsdiv', 'sharebox_260x60', 'ifr', 'jwplayer', 'jwp',
        'pubdate', 'adunit', 'adwidth', 'www', 'bit.ly/2jpexyr', 'googletag.cmd.push', 'googletag.display', '\'div-gpt-ad-1415299254516-0 \'',
        'open', 'opening', 'opens', 'opened', 'alabama', 'loading', 'email', 'advertise', 'apps', 'closed', 'help', 'publish',
        'rendered', 'undefined', 'adsdiv', 'reloadcount', 'adsdiv.reloadcount', 'window.orignetid', '\'undefined', 'window.origadsplid', 'needsrecovery',
        'io_c3sd.ads', 'elem', 'box', 'box.offsetheight', 'box.style.marginright', 'marginright', 'ad_sharebox_260x60', 'trump',
        'russia', 'fox', 'alerts', 'facebook', 'reuters', 'rsize', 'i+= resize /scr+ipt', 'ipt', 'scr', 'script', 'window.adsetsynccalled',
        'adsetsynccalled', 'recoveryid', 'enablequeue', 'slotrenderended', 'beast', 'daily beast', 'clicking', 'subscribed', 'korea', 'north korea', 'south korea',
        'breitbart', 'epa', 'labelmapping', 'comey', 'scotus', 'republicans', 'cohen', 'iran', 'syria', 'eagles', 'april', 'colorado', 'fbi',
        'haspel', 'wedding', 'puerto', 'rico', 'puerto rico', 'huffpost', 'donald', 'don', 'nra', 'sachs', 'kelly', 'facebook', 'mohammed',
        'div-gpt-ad-1415299254516-0 ', '\'div-gpt-ad-1415299254516-0 \'', '\'slotrenderended \'', 'slotrenderended', 
        '\'.single-post # div-gpt-ad-1415299254516-0 \'', 'slotrenderended', 'ldadinit', 'roseanne', 'israel', 'gaza', 
        'china', 'beijing', 'samantha', 'bee', 'valierie', 'subscribe', 'prelimmonth', 'ivanka', 'documentcloud', 'melania', 
        '(', ')', '-', ',', '.', '!', '"', '\'', 'var', "\'div-gpt-ad-1415299254516-0", 'div-gpt-ad-1415299254516-0', '\'.single-post'
        '\'slotrenderended', 'funct', '.contents', 'googletag.pubads', "\'slotrenderended", "\'.single-post", "'div-gpt-ad-1403197269028-0"
        , 'line-height', '>', 'adtech-adspot', '//', '/style', 'overrid', 'window.adsetplid', 'adid', '||', 'adtech_call_typ',
        'by_request', 'adtech_call_typ', 'iframe_proxy', 'ifr.offsetwidth', "ifr.offsetheight", "'jquery", "'readytorecover",
        'args', "waitforglobal", 'arg', "elem.contains", "'reloadad", 'els', "'adtech", 'refreshr', "'none", 'collapsed', 
        'fc', 'f', 'ajax', 'needsrecoveri', 'i+=', '&', '<', '=', 'http', 'https', "'http", "'http", "'script", 'twitter', 
        '/.test', "'https", 'newsletter-inline-widget', 'margin-bottom', '15px', 'font-weight', 'font-size', '12pt', 'div.mc-field-group',
        'padding-bottom', 'padding-right', 'input.mc-input', '.newsletter-inline-widget', 'font-family', '.wpcf7-form-control.wpcf7-text',
        '7e7e7e', 'arial', '.wpcf7', '.wpcf7-form-control.wpcf7-text', '.newsletter.widget__contain', '.wpcf7-form-control.wpcf7-submit',
        'box-shadow', 'text-shadow', 'letter-spacing', '.newsletter.widget__wrap', '.newsletter.widget__head', 'text-transform', 
        'max-width', 'td.first', 'padding-left', 'border-radius', 'hr.divid', 'p.subtext','==typeof', '.gettime', 'window.outerwidth',
        'rcel.async', '/**', 'url', "'data-timestamp", '+new', 'd.head', 'javascript', 'php_widget-140', 'php_widget-104', '.today-on-the-show-cont',
        '.alignleft', 'margin-right', 'margin-top', '.alignright', "'/wp-content/uploads/static/tots.html", "'.today-on-the-show-content",
        "'.today-on-the-show-cont", '.html', 'img.hero-ad-speci', 'fa', 'fa-chevron-down', 'usercollapsetext', 'newsletter-side-widget', 
        'ul.stansberry-form', 'text-align', 'margin-left:0', 'margin-bottom:5px', 'border-top', '-webkit-border-radius', '-khtml-border-radius',
        'border-width', 'border-style', 'border-right', 'padding-left:8px', '.textsiz', 'margin-top:5px', 'ul.links_list', 'img.breaking_imag', 
        'adtech_sharebox_260x60', 'advertisement', '===', 'kraken__adblock.active', 'i=0', 'id=', 'style=', '//www.documentcloud.org/documents/4434037-hhrg-115-if00-wstate-zuckerbergm-20180411.js', 
        '//assets.documentcloud.org/documents/4434037/hhrg-115-if00-wstate-zuckerbergm-20180411.pdf', 'br', 'lt', '//assets.documentcloud.org/documents/4434037/hhrg-115-if00-wstate-zuckerbergm-20180411.txt',
        '//www.documentcloud.org/documents/4425618-van-der-zwaan.js', '//assets.documentcloud.org/documents/4425618/van-der-zwaan.pdf', '//assets.documentcloud.org/documents/4425618/van-der-zwaan.txt',
        'subscrib', 'window.datawrapper', '.embeddeltas', 'datawrapper-height', '=typeof', '==', 'div-gpt-ad-inline_1_mobile', '.jw-player-contain', "'jwp",
        '.playlist', 'episode.title', 'ns_st_st=\\', 'ns_st_pu=\\', 'ns_st_pr=\\', 'ns_st_ep=\\', 'ns_st_ia=\\', 'div-gpt-ad-native_mobile', 'div-gpt-ad-inline_2_mobile',
        'div-gpt-ad-inline_3_mobile', 'ns_st_sn=\\', 'ns_st_en=\\', 'ns_st_ge=\\', 'ns_st_ce=\\', 'ns_st_ddt=\\', 'ns_st_tdt=\\', 'div-gpt-ad-native', 'div-gpt-ad-inline_4_mobile',
        '\\/\\/www.nationalreview.com\\/wp-json\\/wp\\/v2\\/slideshow\\/596990', '\\u0000*\\u0000links', '\\/\\/www.nationalreview.com\\/wp-json\\/wp\\/v2\\/media', '\\/\\/www.nationalreview.com\\/wp-json\\/wp\\/v2\\/categories',
        '\\/\\/api.w.org\\/term', '\\/\\/www.nationalreview.com\\/wp-json\\/wp\\/v2\\/tags', '\\/\\/www.nationalreview.com\\/photos\\/prince-harry-meghan-markle-royal-wedding\\/', '2fwww.nationalreview.com', 'text=harri', '2fprince-harry-meghan-markle-royal-wedding',
        '\\/\\/i1.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-26.jpg', '\\/\\/i0.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-8.jpg',
        '\\/\\/i1.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-37.jpg', '\\/\\/i0.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-110.jpg',
        '\\/\\/i2.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-111.jpg', '\\/\\/i1.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-45.jpg',
        '\\/\\/i2.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-38.jpg', '\\/\\/i2.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-44.jpg',
        '\\/\\/i1.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-7.jpg', '\\/\\/i2.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-12.jpg', 
        '\\/\\/i2.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-11.jpg', '\\/\\/i1.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-43.jpg', 
        '\\/\\/i0.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-21.jpg', '\\/\\/i1.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-18.jpg',
        '\\/\\/i1.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-22.jpg', '\\/\\/i0.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-116.jpg',
        '\\/\\/i0.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-42.jpg', '\\/\\/i0.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-14.jpg',
        '\\/\\/i0.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-39.jpg', '\\/\\/i1.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-54.jpg',
        '\\/\\/i1.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-112.jpg', '\\/\\/i2.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-106.jpg',
        '\\/\\/i1.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-25.jpg', '\\/\\/i2.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-46.jpg',
        '\\/\\/i0.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-24.jpg', '\\/\\/i2.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-29.jpg',
        '\\/\\/i2.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-48.jpg', '\\/\\/i0.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-107.jpg',
        '\\/\\/i0.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-47.jpg', '\\/\\/i2.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-33.jpg',
        '\\/\\/i1.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-51.jpg', 'google_ad_cli', 
        'adunit.style.display', '.length', 'google_ad_width', 'google_ad_height', 'adunit.style.margin', 'adunit.style.textalign', 'adunit.classnam',
        '//www.activistpost.com/2018/06/israeli-selling-surveillance-systems-governments-around-world.html', "'theactivistpost", '.innerhtml', 'ld-ajs', 'w.ldadinit=w.ldadinit||', '//www.activistpost.com/2018/06/google-quits-drone-program-u-s-navy-wants-drone-motherships-with-help-of-a-i.html',
        '302px', '422px', '300px', '0px', '12px', '60px', '25px', '250px', '3px', '40px', '1px', '10px','titl', '0\\', '//www.activistpost.com/2018/04/us-bombs-syria-to-cover-up-lack-of-evidence-on-chem-attacks-discredits-own-claims-by-doing-so.html', 
        '//www.activistpost.com/2018/05/red-nose-day-recognizes-kids-living-in-poverty-article-the-rich-get-smart-the-poor-get-technology-the-new-digital-divide-in-school-choice-highlights-part-of-the-problem.html', 
        'firstscript', 'dsqlocal', "'trackback_url", 'disqus_shortnam', 'fb-extra-h2', 'prelimd', 'resize', 'scr+ipt', 'paul', 'watson']
        regex = re.compile(r"\S+www.nationalreview.com\S+")
        linkstr = re.findall(regex, str(werdz))
        say_no.append(linkstr)
        #regex2 = re.compile(r"\\\S+")
        #linkstr2 = re.findall(regex2, str(werdz))
        #say_no.append(linkstr2)

        werdz = ([s[-2] for s in werdz if s[-2] not in say_no and 'wp.com' not in s[-2] and 'facebook.com' not in s[-2] and
                 '1897954795849722' not in s[-2] and '999999' not in s[-2] and "'100" not in s[-2] and 'prelimmonth' not in s[-2]
                 and '5g' not in s[-2] and 'scr+ipt' not in s[-2] and 'documentcloud.org' not in s[-2] and 'www.' not in s[-2]
                 and '10 10' not in s[-2] and 'labelmap' not in s[-2] and 'finald' not in s[-2] and '23390304' not in s[-2]
                 and 'box.' not in s[-2] and 'label.' not in s[-2] and '.style' not in s[-2] and '10 100' not in s[-2]])
        
        #check cleaned text line from function above
        no_nouns.append(' '.join(werdz))
        #no_nouns.append(werdz)
    return no_nouns

In [91]:
cleaned_text = clean_text(true_df.body)

In [92]:
how_can_this_happen = drop_nouns(cleaned_text)

In [52]:
'''tfidf = pickle.load(open("pickle_files/tfidf.pkl", "rb" ))'''

'tfidf = pickle.load(open("pickle_files/tfidf.pkl", "rb" ))'

In [53]:
'''cor_tfidf = tfidf.transform(how_can_this_happen)'''

'cor_tfidf = tfidf.transform(how_can_this_happen)'

In [54]:
'''corpus_lsa = pickle.load(open("pickle_files/corpus_lsa.pkl", "rb" ))'''

'corpus_lsa = pickle.load(open("pickle_files/corpus_lsa.pkl", "rb" ))'

In [55]:
'''%%time
lsa = pickle.load(open("pickle_files/lsa.pkl", "rb" ))'''

'%%time\nlsa = pickle.load(open("pickle_files/lsa.pkl", "rb" ))'

In [93]:
tfidf = TfidfVectorizer(ngram_range=(1, 4),  
                                   stop_words='english', 
                                   #token_pattern="\\b[a-z][a-z]+\\b",
                                   lowercase=True,
                                   max_df = 0.6)
cor_tfidf = tfidf.fit_transform(how_can_this_happen)

In [94]:
lsa = TruncatedSVD(140, algorithm = 'arpack')
corpus_lsa = lsa.fit_transform(cor_tfidf)
corpus_lsa = Normalizer(copy=False).fit_transform(corpus_lsa)

In [95]:
sum(lsa.explained_variance_ratio_)

0.09813519926607474

In [96]:
terms = tfidf.get_feature_names()

In [60]:
'''km = pickle.load(open("pickle_files/km.pkl", "rb" ))'''

'km = pickle.load(open("pickle_files/km.pkl", "rb" ))'

In [97]:
km = KMeans(n_clusters=140)
clusts = km.fit_predict(corpus_lsa)

In [98]:
len(clusts)

4092

In [99]:
true_df['clusters'] = clusts

In [100]:
true_df.tail()

,body,num_source,source,title,clusters
226,\n\n\nFox News\n\n\n\n\n\n\n\n\n\n\nU.S.\nWorl...,7.0,Fox News,Trump blames Sessions for 'Russian Witch Hunt ...,55
228,\n\n\nFox News\n\n\n\n\n\n\n\n\n\n\nU.S.\nWorl...,7.0,Fox News,"Conservative groups, congressional Republicans...",55
230,\n\n\nFox News\n\n\n\n\n\n\n\n\n\n\nU.S.\nWorl...,7.0,Fox News,Dems struggling to flip key California House s...,55
232,\n\n\nFox News\n\n\n\n\n\n\n\n\n\n\nU.S.\nWorl...,7.0,Fox News,"Blackburn, Bredesen lead candidates vying for ...",55
234,\n\n\nFox News\n\n\n\n\n\n\n\n\n\n\nU.S.\nWorl...,7.0,Fox News,"Former Obama official, liberal activists share...",55


In [101]:
true_df.clusters.value_counts()

2      136
50     116
55     113
129    107
29      89
90      82
115     78
108     71
37      67
19      62
73      62
49      60
42      57
27      57
16      56
88      54
78      51
112     51
53      50
11      47
133     45
106     45
64      44
21      43
137     43
96      43
117     43
18      43
26      43
8       41
      ... 
14      13
125     12
105     12
57      12
91      12
87      11
83      11
46      11
59      11
45      10
116     10
124     10
139      9
82       9
128      9
102      9
60       8
84       7
32       6
35       6
99       6
121      6
113      5
13       5
136      4
12       4
44       3
123      3
75       2
107      2
Name: clusters, Length: 140, dtype: int64

In [71]:
dt = pickle.load(open("pickle_files/dt.pkl", "rb" ))


In [66]:
os.system('say "Your move.....Commander"')

0

In [102]:
X = true_df.clusters
y = true_df.source
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = X_train.reshape(-1,1)
X_test = X_test.reshape(-1,1)


/Users/NickThomas/Desktop/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  after removing the cwd from sys.path.
/Users/NickThomas/Desktop/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """


In [104]:
dt = DecisionTreeClassifier(max_depth=25)
dt.fit(X_train, y_train)
print(dt.score(X_test, y_test))
print(dt.score(X_train, y_train))

0.5702075702075702
0.6324472960586618


In [112]:
dt.predict(125)

array(['Daily Beast'], dtype=object)

In [106]:
pipe = Pipeline(steps=[('tfidf', tfidf), ('lsa', lsa), ('Normalizer', Normalizer), ('km', km)])



In [107]:
cleaned_text = clean_text(book_df.body)

In [108]:
let_do_this = drop_nouns(cleaned_text)

In [115]:
le = [how_can_this_happen]
new_clust = pipe.transform(le)

AttributeError: 'list' object has no attribute 'lower'

In [111]:
let_do_this

['erik found embroil special oper social specialist potenti pro-trump social influenc russian kiril dmitriev period—a set administr rais administration—quest investigating lobbi afghanistan prosecuted relianc privat afghan wa recent subject in-depth hi secur frontier servic quot critic say ’ run counter american ’ first on-the-record lightli there ’ ’ lead-up could tell you ’ v intens certainli wild-ey spoken voluntarili special plenti there ’ take serious think it ’ best keep now add much inaccurate confid finish abl put distract told hous seychel with dmitriev unplanned abc georg nader make abc relat there gop there ’ it ’ democrat shred would hire say is there ’ wrong get wrong ever get right spoken cooper special erik princ think ’ postur toward be absolutely i ’ said before stalin defeat german nazi socialist islam think good could even start russian ukrain be certainly enemi even nato mean remember 400,000 american twenty-two million russian nazi russian unfriendli may 1940. ’ th